In [1]:
!pip install fastai --upgrade

     |████████████████████████████████| 204kB 33.3MB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [ ]:
import torch
import math
import glob
import cv2
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import datasets, io, transforms
from torch.utils.data import Dataset, DataLoader, Subset
from fastai.data.external import untar_data, URLs
from PIL import Image, ImageCms
from sklearn.model_selection import train_test_split
from skimage.color import rgb2lab, lab2rgb
from skimage.io import imread
from torchvision.utils import save_image
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
# import tensorflow as tf
import numpy as np
import os
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
coco_imgs = untar_data(URLs.COCO_SAMPLE)
coco_imgs = str(coco_imgs) + "/train_sample"

# Data Loading

In [ ]:
# class ColorDataset(Dataset):
#     def __init__(self, root_path="", split="train", transform=None):
#         self.split = split
#         self.root_path = root_path
#         self.transform = transform

#         self.srgb_p = ImageCms.createProfile("sRGB")
#         self.lab_p = ImageCms.createProfile("LAB")
#         self.rgb2lab = ImageCms.buildTransformFromOpenProfiles(self.srgb_p, self.lab_p, "RGB", "LAB")

#         self.paths = os.listdir(self.root_path)

#         self.train_split, self.val_split, self.test_split = torch.utils.data.random_split(
#             self.paths,
#             [
#                 math.floor(0.8 * len(self.paths)),
#                 math.floor(0.1 * len(self.paths)),
#                 len(self.paths) -  math.floor(0.1 * len(self.paths)) -  math.floor(0.8 * len(self.paths))
#             ],
#             generator=torch.Generator().manual_seed(442)
#         )
#         self.splits = {
#             "train": self.train_split,
#             "val": self.val_split,
#             "test": self.test_split,
#         }


#     def load_images(self):
#         return self.df.img_path.apply(self.load_image)
    
    
#     def load_image(self, img_path):
#         img = imread(
#             os.path.join(
#                 self.root_path,
#                 img_path
#             ),
#             # io.image.ImageReadMode.RGB
#         )

#         img = rgb2lab(img)
#         return torch.tensor(img)
            
#     def __len__(self):
#         return len(self.splits[self.split])

#     def __getitem__(self, idx):
#         lab_img = self.load_image(self.splits[self.split][idx])
#         H, W, C = lab_img.shape
#         lab_img = lab_img.view(C, H, W)
#         if self.transform:
#             lab_img = self.transform(lab_img)
#         print(lab_img[0].shape)
#         return lab_img[0], lab_img[1:]
class ColorDataset(Dataset):
    def __init__(self, root_path=coco_imgs, transform=None, mode='test'):
        self.train_transforms = transforms.Compose([
          transforms.Resize((256, 256)),
          transforms.RandomHorizontalFlip()
        ])
        self.other_transforms = transforms.Resize((256, 256))
        self.mode = mode
        
        self.paths = root_path
    
    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB")
        if (self.mode == 'train'):
          img = self.train_transforms(img)
        else:
          img = self.other_transforms(img)
        img = np.array(img)
        img_lab = rgb2lab(img).astype("float32") # Converting RGB to L*a*b
        img_lab = transforms.ToTensor()(img_lab)
        L = img_lab[[0], ...] / 50. - 1. # Between -1 and 1
        ab = img_lab[[1, 2], ...] / 110. # Between -1 and 1
        
        return L, ab
    def __len__(self):
        return len(self.paths)

# Generator

In [ ]:
def encoder(in_channels=0,out_channels=0,first=False):
  if first:
    return nn.Sequential(
        nn.Conv2d(in_channels,out_channels, kernel_size=4,stride=2,padding=1),
        nn.LeakyReLU(0.2)
    )
  else:
    return nn.Sequential(
        nn.Conv2d(in_channels,out_channels, kernel_size=4,stride=2,padding=1),
        nn.BatchNorm2d(out_channels),
        nn.LeakyReLU(0.2)
    )

In [ ]:
def decoder(in_channels=0,out_channels=0,dropout = False):
  if dropout:
    return nn.Sequential(
        nn.ConvTranspose2d(in_channels,out_channels, kernel_size=4,stride=2,padding=1),
        nn.BatchNorm2d(out_channels),
        nn.Dropout2d(),
    )
  else:
    return nn.Sequential(
      nn.ConvTranspose2d(in_channels,out_channels, kernel_size=4,stride=2,padding=1),
      nn.BatchNorm2d(out_channels),
  )

In [ ]:
class Generator(torch.nn.Module):
  def __init__(self):
    super(Generator, self).__init__()

    self.encoder1 = encoder(in_channels=1,   out_channels=64,first = True)
    self.encoder2 = encoder(in_channels=64,  out_channels=128)
    self.encoder3 = encoder(in_channels=128, out_channels=256)
    self.encoder4 = encoder(in_channels=256, out_channels=512)
    self.encoder5 = encoder(in_channels=512, out_channels=512)
    self.encoder6 = encoder(in_channels=512, out_channels=512)
    self.encoder7 = encoder(in_channels=512, out_channels=512)

    self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels=512,out_channels=512,kernel_size=4,stride=2,padding=1),
        nn.ReLU()
    )

    self.decoder1 = decoder(in_channels=512, out_channels=512, dropout=True)
    self.decoder2 = decoder(in_channels=1024, out_channels=512, dropout=True)
    self.decoder3 = decoder(in_channels=1024, out_channels=512, dropout=True)
    self.decoder4 = decoder(in_channels=1024, out_channels=512)
    self.decoder5 = decoder(in_channels=1024, out_channels=256)
    self.decoder6 = decoder(in_channels=512, out_channels=128)
    self.decoder7 = decoder(in_channels=256, out_channels=64)

    self.conv2 = nn.Sequential(
        nn.ConvTranspose2d(in_channels = 128, out_channels = 2,kernel_size=4,stride=2,padding=1),
        nn.Tanh()
    )

  def encode(self,x):
    skips = []
    #print(x.shape)
    x = self.encoder1(x)
    skips.append(x)
    #print(x.shape)
    x = self.encoder2(x)
    skips.append(x)
    #print(x.shape)

    x = self.encoder3(x)
    skips.append(x)
    #print(x.shape)

    x = self.encoder4(x)
    skips.append(x)
    #print(x.shape)

    x = self.encoder5(x)
    skips.append(x)
    #print(x.shape)

    x = self.encoder6(x)
    skips.append(x)
    #print(x.shape)

    x = self.encoder7(x)
    skips.append(x)
    #print(x.shape)

    x = self.conv1(x)
    #print(x.shape)

    return x, skips

  def decode(self, x, skips):
    #print('decoding',x.shape)
    x = self.decoder1(x)
    x = F.relu(x)
    x = torch.cat([x,skips[6]],1)
    #print(x.shape)

    x = self.decoder2(x)
    x = torch.cat([x,skips[5]],1)
    x = F.relu(x)

    x = self.decoder3(x)
    x = torch.cat([x,skips[4]],1)
    x = F.relu(x)

    x = self.decoder4(x)
    x = torch.cat([x,skips[3]],1)
    x = F.relu(x)

    x = self.decoder5(x)
    x = torch.cat([x,skips[2]],1)
    x = F.relu(x)

    x = self.decoder6(x)
    x = torch.cat([x,skips[1]],1)
    x = F.relu(x)

    x = self.decoder7(x)
    x = torch.cat([x,skips[0]],1)
    x = F.relu(x)

    return x

  def forward(self,x):
    x, skips = self.encode(x)
    out = self.decode(x,skips)
    return self.conv2(out)


In [ ]:
def test():
    x = torch.randn((2,1,256,256))
    g = Generator()
    print(g.eval())
    preds = g(x)
    print(preds.shape)

In [ ]:
test()

# Discriminator

In [ ]:
class CNNBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels, stride=2):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,out_channels=out_channels, kernel_size=4, stride=stride, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
        )
    def forward(self, x):
        return self.conv(x)

In [ ]:
class Discriminator(torch.nn.Module):
    def __init__(self, in_channels=3, dims=[64,128,256,512]):
        super().__init__()
        self.first = nn.Sequential(
            nn.Conv2d(3,dims[0], kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
        )

        layers = []
        in_channels = dims[0]
        for dim in dims[1:]:
            layers.append(
                CNNBlock(in_channels, dim, stride=1 if dim == dims[-1] else 2), # stride only differs for last cnn block
            )
            in_channels = dim
        
        layers.append(
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=1)
        )
        
        self.model = nn.Sequential(*layers)

    def forward(self, x, y):
        x = torch.cat([x,y], dim=1)
        #print(x.shape)
        x = self.first(x)
        return self.model(x)

In [ ]:
def test():
    x = torch.randn((1,1,256,256))
    y = torch.randn((1,2,256,256))
    d = Discriminator()
    print(d.eval())
    preds = d(x,y)
    print(preds.shape)

In [ ]:
test()

# Training Utilities

In [ ]:
def lab_to_rgb(L, ab):
    """
    Takes a batch of images
    """
    
    L = (L + 1.) * 50.
    ab = ab * 110.
    Lab = torch.cat([L, ab], dim=1).permute(0, 2, 3, 1).cpu().numpy()
    rgb_imgs = []
    for img in Lab:
        img_rgb = lab2rgb(img)
        rgb_imgs.append(img_rgb)
    return np.stack(rgb_imgs, axis=0)

# Training


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
model_save_name = 'classifier.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
output_path = "/content/gdrive/My Drive/GAN_output/" 

In [ ]:
d = Discriminator(in_channels=3).to(device)
g = Generator().to(device)
opt_d = optim.Adam(d.parameters(), lr=2e-4, betas=(0.5, 0.999))
opt_g = optim.Adam(g.parameters(), lr=2e-4, betas=(0.5, 0.999))

In [ ]:
BCE = nn.BCEWithLogitsLoss()
L1_loss = nn.L1Loss()

trans = transforms.Compose([
  transforms.Resize((256, 256)),
  transforms.RandomHorizontalFlip()
])

# The batch size we'll use for training
batch_size = 16
num_epochs = 100
L1_LAMBDA = 100 #used in paper
val_split = 0.1

paths = glob.glob(coco_imgs + "/*.jpg")
np.random.seed(123) # Seeding for reproducible results
paths_subset = np.random.choice(paths, 12_000, replace=False) # Randomly choosing 10,000 images
rand_idxs = np.random.permutation(12_000) # Shuffling the indexes
train_idxs = rand_idxs[:8000]
val_idxs = rand_idxs[8000:10000]
test_idxs = rand_idxs[10000:12000]
train_paths = paths_subset[train_idxs] 
val_paths = paths_subset[val_idxs]
test_paths = paths_subset[test_idxs]

train_data = ColorDataset(train_paths, transform=trans, mode='train')
train_dataloader = DataLoader(train_data, batch_size=batch_size, num_workers=4, drop_last=True) #num_workers and shuffle?

g_scale = torch.cuda.amp.GradScaler()
d_scale = torch.cuda.amp.GradScaler()

val_data = ColorDataset(val_paths, transform=trans, mode='val')
val_dataloader = DataLoader(val_data, batch_size=8, num_workers=4, drop_last=True) #num_workers and shuffle?

In [ ]:
# checkpoint = torch.load("./model.pt")
# d.load_state_dict(checkpoint['discriminator_state'])
# g.load_state_dict(checkpoint['generator_state'])
# opt_d.load_state_dict(checkpoint['discriminator_optim_state'])
# opt_g.load_state_dict(checkpoint['generator_optim_state'])
# epoch = checkpoint['epoch']

In [ ]:
G_losses = []
D_losses = []

for epoch in range(num_epochs):
    loop = tqdm(train_dataloader, leave=True)

    for idx, (l, ab) in enumerate(loop):
        l, ab = l.to(device), ab.to(device)

        #Discriminator train
        
        #l = l.to(torch.float32)
        ab_fake = g(l)
        D_real = d(l, ab)
        D_fake = d(l, ab_fake.detach())

        D_real_l = BCE(D_real, torch.ones_like(D_real))
        D_fake_l = BCE(D_fake, torch.zeros_like(D_fake))

        D_loss = (D_real_l + D_fake_l) / 2 #similar to cyclegan divide by 2, reason being trains discriminator slower relative to generator according to paper, doesnt make much sense?
            
        #loss.backward(retain_graph=True)
        opt_d.zero_grad()
        D_loss.backward()
        opt_d.step()

        #Generator train
        
        D_fake = d(l, ab_fake)
        G_fake_l = BCE(D_fake, torch.ones_like(D_fake))
        L1 = L1_loss(ab_fake, ab) * L1_LAMBDA
        G_loss = G_fake_l + L1
        
        opt_g.zero_grad()
        G_loss.backward()
        opt_g.step()
    torch.save({
        "discriminator_state": d.state_dict(),
        "generator_state": g.state_dict(),
        "discriminator_optim_state": opt_d.state_dict(),
        "generator_optim_state": opt_g.state_dict(),
        "epoch": epoch,
    }, "./model.pt")
    print("Discriminator loss:", D_loss)
    D_losses.append(D_loss)
    print("Generator loss:", G_loss)
    G_losses.append(G_loss)
    l_, ab_ = next(iter(val_dataloader))
    l_, ab_ = l_.to(device), ab_.to(device)
    g.eval()
    with torch.no_grad():
        fake_color = g(l_).detach()
        real_color = ab_
        L = l_
        fake_imgs = lab_to_rgb(L, fake_color)
        real_imgs = lab_to_rgb(L, real_color)
        fig = plt.figure(figsize=(15, 8))
        for i in range(5):
            ax = plt.subplot(3, 5, i + 1)
            ax.imshow(L[i][0].cpu(), cmap='gray')
            ax.axis("off")
            ax = plt.subplot(3, 5, i + 1 + 5)
            ax.imshow(fake_imgs[i])
            ax.axis("off")
            ax = plt.subplot(3, 5, i + 1 + 10)
            ax.imshow(real_imgs[i])
            ax.axis("off")
        plt.show()
    g.train()
    

In [ ]:
torch.save({
    "discriminator_state": d.state_dict(),
    "generator_state": g.state_dict(),
    "discriminator_optim_state": opt_d.state_dict(),
    "generator_optim_state": opt_g.state_dict(),
    "epoch": epoch,
}, path)

In [ ]:
test_data = ColorDataset(test_paths, transform=trans, mode='test')
test_dataloader = DataLoader(test_data, batch_size=8, num_workers=4, drop_last=True) #num_workers and shuffle?

In [ ]:
checkpoint = torch.load(path)
d.load_state_dict(checkpoint['discriminator_state'])
g.load_state_dict(checkpoint['generator_state'])
opt_d.load_state_dict(checkpoint['discriminator_optim_state'])
opt_g.load_state_dict(checkpoint['generator_optim_state'])
epoch = checkpoint['epoch']

In [ ]:
psnr_vals = []

j = 0

for idx, (L, ab) in enumerate(test_dataloader):
    L, ab = L.to(device), ab.to(device)
    fake_color = g(L).detach()
    real_color = ab
    #print(fake_color.shape)
    if (j % 50):
        fake_imgs = lab_to_rgb(L, fake_color)
        real_imgs = lab_to_rgb(L, real_color)
        for i in range(len(fake_imgs)):
            f = plt.figure()
            ax1 = f.add_subplot(1,2, 1)
            plt.imshow(fake_imgs[i])
            ax2= f.add_subplot(1,2, 2)
            plt.imshow(real_imgs[i])

            ax1.set_title('Fake')
            ax2.set_title('Real')

            plt.show(block=True)
    psnr_vals.append(cv2.PSNR(fake_imgs[0], real_imgs[0]))
    j += 1

In [ ]:
print('Average PSNR: ', sum(psnr_vals) / len(psnr_vals))

In [ ]:
plt.figure(figsize=(9, 3))
plt.subplot(1, 2, 1)
plt.plot(G_losses)
plt.xlabel("Epoch")
plt.ylabel("Generator Loss")
plt.subplot(1, 2, 2)
plt.plot(D_losses)
plt.xlabel("Epoch")
plt.ylabel("Discriminator Loss")
plt.show

In [ ]:
"Random image color testing"

psnr_vals_random = []
i = 0

for idx, (L, ab) in enumerate(test_dataloader):
    L, ab = L.to(device), ab.to(device)
    fake_color = torch.randn((8,2,256,256)).to(device)
    real_color = ab
    #print(fake_color.shape)
    if (i % 100 == 0):
        fake_imgs = lab_to_rgb(L, fake_color)
        real_imgs = lab_to_rgb(L, real_color)
        for i in range(len(fake_imgs)):
          f = plt.figure()
          ax1 = f.add_subplot(1,2, 1)
          plt.imshow(fake_imgs[i])
          ax2= f.add_subplot(1,2, 2)
          plt.imshow(real_imgs[i])

          ax1.set_title('Fake')
          ax2.set_title('Real')

          plt.show(block=True)
    psnr_vals_random.append(cv2.PSNR(fake_imgs[0], real_imgs[0]))
    i += 1

In [ ]:
print('Average PSNR Random: ', sum(psnr_vals_random) / len(psnr_vals_random))